<div style="background-color: #0000FF; color: white; padding: 10px; border-radius: 5px;">
  <h1>Team Info</h1>
</div>

## Team name: Turbo Adventure
## Team members: Akshay, Louis, Reno, Gabrijel


<div style="background-color: #0000FF; color: white; padding: 10px; border-radius: 5px;">
  <h1>1 Feature Engineering</h1>
</div>

## 1.1 Imports

In [1]:
import os
import librosa
import numpy as np
import pandas as pd
from librosa.feature import rhythm  
from IPython.display import display  

## 1.2 Defining functions

In [2]:
# ChatGPT, 2024, Prompt 1: "welke geluidsfeatures zijn er voor clustering van geluidsfragmenten"
# ChatGPT, 2024, Prompt 2: "geef per feature aan welke library ik daarvoor kan gebruiken"
# ChatGPT, 2024, Prompt 3: "laat voorbeeld code zien hoe deze features worden toegepast op een geluidsfragment, en vervolgens in een df wordt gezet"
# ChatGPT, 2024, Prompt 3: "hoe pas ik oop toe op deze code"
# Link: https://chatgpt.com/share/677ee38d-fb54-8001-a50a-9856d52e22c9

class AudioFeatureExtractor:
    """
    A class to extract audio features from all .wav files in a given folder.
    """

    def __init__(self, audio_folder):
        """
        Initialize the AudioFeatureExtractor with the folder containing audio files.
        """
        self.audio_folder = audio_folder
        self.features_list = []

    def process_audio_files(self):
        """
        Iterate through all .wav files in the folder and extract features.
        """
        for file_name in os.listdir(self.audio_folder):
            if file_name.endswith(".wav"):  # Only process .wav files
                file_path = os.path.join(self.audio_folder, file_name)
                self._extract_features(file_name, file_path)

    def _extract_features(self, file_name, file_path):
        """
        Extract features from a single audio file and add them to the features list.
        """
        try:
            # Load the audio file
            y, sr = librosa.load(file_path, sr=None)

            # Compute features
            features = {
                "File Name": file_name,
                "Zero Crossing Rate": np.mean(librosa.feature.zero_crossing_rate(y)[0]),
                "RMS Amplitude": np.mean(librosa.feature.rms(y=y)[0]),
                "Spectral Centroid": np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)[0]),
                "Spectral Bandwidth": np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]),
                "Spectral Roll-Off": np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)[0]),
                "Spectral Flatness": np.mean(librosa.feature.spectral_flatness(y=y)[0]),
                "MFCC1": np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)[0]),
                "MFCC2": np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)[1]),
                "Chroma Features": np.mean(librosa.feature.chroma_cqt(y=y, sr=sr)),
                "Tempo (BPM)": rhythm.tempo(y=y, sr=sr)[0],
            }

            # Append features to the list
            self.features_list.append(features)

        except Exception as e:
            print(f"Error processing {file_name}: {e}")

    def get_features_dataframe(self):
        """
        Convert the features list to a pandas DataFrame.
        """
        return pd.DataFrame(self.features_list)

## 1.3 Extract Features & Create Dataframe

In [3]:
# ChatGPT, 2024, Prompt 1: "welke geluidsfeatures zijn er voor clustering van geluidsfragmenten"
# ChatGPT, 2024, Prompt 2: "geef per feature aan welke library ik daarvoor kan gebruiken"
# ChatGPT, 2024, Prompt 3: "laat voorbeeld code zien hoe deze features worden toegepast op een geluidsfragment, en vervolgens in een df wordt gezet"
# ChatGPT, 2024, Prompt 3: "hoe pas ik oop toe op deze code"
# Link: https://chatgpt.com/share/677ee38d-fb54-8001-a50a-9856d52e22c9

# Define the folder containing audio files
audio_folder_path = r"muziek-genre-clustering-24-25\unlabeled"

# Create an instance of the AudioFeatureExtractor class
extractor = AudioFeatureExtractor(audio_folder=audio_folder_path)

# Process the audio files
extractor.process_audio_files()

# Get the features as a DataFrame
audio_features_df = extractor.get_features_dataframe()

# Display the DataFrame
display(audio_features_df)

,File Name,Zero Crossing Rate,RMS Amplitude,Spectral Centroid,Spectral Bandwidth,Spectral Roll-Off,Spectral Flatness,MFCC1,MFCC2,Chroma Features,Tempo (BPM)
0,m00003.wav,0.120116,0.136221,2254.606986,2071.028440,4381.532206,0.009073,-82.511116,97.341721,0.571940,135.999178
1,m00012.wav,0.178108,0.217475,2908.300131,2286.252592,5405.816551,0.026373,-1.913298,72.686157,0.533881,103.359375
2,m00013.wav,0.123423,0.029083,1952.607933,1865.996047,3779.909832,0.002211,-287.603699,101.758171,0.385155,95.703125
3,m00043.wav,0.126073,0.100411,2384.634958,2072.759900,4584.132502,0.008830,-120.147491,91.317215,0.557109,135.999178
4,m00044.wav,0.052088,0.018833,790.507005,900.409298,1159.412273,0.000153,-437.604401,170.931534,0.332215,107.666016
...,...,...,...,...,...,...,...,...,...,...,...
100,m00971.wav,0.091590,0.038885,1525.791709,1588.217850,2733.184659,0.006208,-246.158646,149.428665,0.414986,107.666016
101,m00973.wav,0.161797,0.259782,3669.810467,3295.063699,7876.372739,0.088763,-8.573005,54.295788,0.475779,129.199219
102,m00988.wav,0.167480,0.240724,3941.417697,3361.641125,8260.423162,0.089354,-56.153061,36.448502,0.480985,103.359375
103,m00991.wav,0.044217,0.017138,983.971719,1404.408040,1853.129475,0.000302,-406.846680,163.081345,0.235405,117.453835


## 1.4 Features Explanation

### Zero Crossing Rate

### RMS Amplitude

### Spectral Centroid

### Spectral Bandwidth

### Spectral Roll-Off

### Spectral Flatness

### MFCC1

### MFCC2

### Chroma Features

### Tempo (BPM)

#

<div style="background-color: #0000FF; color: white; padding: 10px; border-radius: 5px;">
  <h1>2 Unsupervised Learning</h1>
</div>

## 2.1

<div style="background-color: #0000FF; color: white; padding: 10px; border-radius: 5px;">
  <h1>3 Findings and Conclusion</h1>
</div>

## 3.1

<div style="background-color: #0000FF; color: white; padding: 10px; border-radius: 5px;">
  <h1>4 Reference List</h1>
</div>

## 4.1

* ChatGPT, 2024, Prompt 1: "welke geluidsfeatures zijn er voor clustering van geluidsfragmenten"
* ChatGPT, 2024, Prompt 2: "geef per feature aan welke library ik daarvoor kan gebruiken"
* ChatGPT, 2024, Prompt 3: "laat voorbeeld code zien hoe deze features worden toegepast op een geluidsfragment, en vervolgens in een df wordt gezet"
* ChatGPT, 2024, Prompt 3: "hoe pas ik oop toe op deze code"